In [7]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import sys
import scipy

the system: 

$$\dot{x} = Ax + Bu$$

where $x$ is the state vector: $$x = \begin{bmatrix} x \\ \dot{x} \\ z \\ \dot{z} \\ \theta \\ \dot{\theta} \end{bmatrix}$$

dynamic equations:
$$\ddot{x} = -\frac{1}{m} F * \sin(\theta) $$
$$\ddot{z} = \frac{1}{m} F * \cos(\theta) - g$$
$$\ddot{\theta} = \frac{1}{I} \tau$$

our goal is to find $B$ matrix. 

Lets try a static case where $\frac{1}{\tau}=k$ is unknown constant. we can add that to the state vector.

so. state vector, $x$ becomes: $$x = \begin{bmatrix} x \\ \dot{x} \\ z \\ \dot{z} \\ \theta \\ \dot{\theta} \\ k \end{bmatrix}$$



# constants

In [4]:
# System Parameters
m = 0.5 # mass
l = 0.2 # distance from rotor to COM
I = 2/5*m*l**2 # Moment of inertia (Iyy) -- assuming drone is a sphere (it is not)
g = 9.81 # gravity constant

# Control Parameters
tau_k_theta = 10 # torque control to stabilize theta
tau_k_thetadot = 20 # torque control to provide thetadot damping
tau_k_x = 0.01 # torque control to control x position
tau_k_xdot = 0.2 # torque control to provide xdot damping
F_k_z = 1000 # Force control to control z position
F_k_zdot = 10000 # Force control to control z position

F0 = m*g

# Noise for measurements: (standard deviations)
v = [0.01, 0.001, 0.2, 0.2]

# Desired trajectory

In [5]:
def desired_trajectory(t):
    theta_des = 0*np.ones_like(t)
    z_des = 1*np.ones_like(t)
    x_des = np.sin(0.01*t)
    return theta_des, x_des, z_des

# Nonlinear continuous time dynamics with control

In [6]:
def control(X, t):
    theta, thetadot, x, xdot, z, zdot = np.ravel(X)
    
    # desired trajectory
    theta_des, x_des, z_des = desired_trajectory(t)
    
    # error
    x_err = x_des - x
    z_err = z_des - z
    
    # control
    tau = -(tau_k_x*x_err - tau_k_xdot*xdot)  + (tau_k_theta*(-theta) - tau_k_thetadot*thetadot)
    F = (F_k_z*z_err - F_k_zdot*zdot)
    
    return F, tau

In [7]:
def f(X, t):
    theta, thetadot, x, xdot, z, zdot = X
    
    # control
    F, tau = control(X, t)
    
    # dynamics
    d_theta = thetadot
    d_thetadot = tau/I
    d_x = xdot
    d_xdot = -F*np.sin(theta)/m
    d_z = zdot
    d_zdot = (F*np.cos(theta)-m*g)/m
    
    d_x = [d_theta, d_thetadot, d_x, d_xdot, d_z, d_zdot]
    
    return d_x

# Run simulation

In [8]:
t = np.arange(0, 1000, 0.1)

x0 = [(np.random.random()-0.5)*1e-3 for i in range(6)]

result = odeint(f, x0, t)
X = result.T

In [19]:
a = np.eye(9)
a[6:,6:] = 0

# np.linalg.matrix_power(a, 20)
c = np.zeros((6,9))
c[:6,:6] = np.eye(6)
c@a

np.linalg.matrix_rank(c@a) == np.linalg.matrix_rank(a)

True